In [9]:
from controller import Imager

In [10]:
imager = Imager()

Found 4 devices
Devices IDs - ['101936', '101934', '101935', '101933']


In [7]:
imager.cams[-1].remote_device.node_map.ExposureAuto.value = 'Off'
imager.update_params(ExposureTime=20000, AcquisitionFrameRate = 5)

AccessException: Node is not writable. : AccessException thrown in node 'AcquisitionFrameRate' while calling 'AcquisitionFrameRate.SetValue()' (file 'FloatT.h', line 87)

In [16]:
PIXELFORMATS = {'8':'Mono8','12':'Mono12Packed'}
PIXELFORMAT = PIXELFORMATS['8']
imager.update_params(ExposureTime=20000, PixelFormat=PIXELFORMAT, AcquisitionFrameRate = 50)

In [18]:
imager._start_acquisitions()

In [19]:
# imager.get_images(show_images=True, save_images=False)
raw_images = imager.capture_sequence(num_frames=50, sleep_seconds=0.2)

cam 101936 captured image of format Mono8 at 2021-06-16 17:55:51.907006
cam 101934 captured image of format Mono8 at 2021-06-16 17:55:51.908007
cam 101935 captured image of format Mono8 at 2021-06-16 17:55:51.909007
cam 101933 captured image of format Mono8 at 2021-06-16 17:55:51.911007
cam 101936 captured image of format Mono8 at 2021-06-16 17:55:52.168678
cam 101934 captured image of format Mono8 at 2021-06-16 17:55:52.187756
cam 101935 captured image of format Mono8 at 2021-06-16 17:55:52.190510
cam 101933 captured image of format Mono8 at 2021-06-16 17:55:52.191760
cam 101936 captured image of format Mono8 at 2021-06-16 17:55:52.440733
cam 101934 captured image of format Mono8 at 2021-06-16 17:55:52.442734
cam 101935 captured image of format Mono8 at 2021-06-16 17:55:52.443732
cam 101933 captured image of format Mono8 at 2021-06-16 17:55:52.445733
cam 101936 captured image of format Mono8 at 2021-06-16 17:55:52.666638
cam 101934 captured image of format Mono8 at 2021-06-16 17:55:52

In [5]:
# imager.get_images(show_images=True, save_images=False)
raw_images = imager.capture_sequence(num_frames=50, sleep_seconds=0.2)

cam 101936 captured image of format Mono8 at 2021-06-16 17:51:26.743861
cam 101934 captured image of format Mono8 at 2021-06-16 17:51:26.833860
cam 101935 captured image of format Mono8 at 2021-06-16 17:51:26.889859
cam 101933 captured image of format Mono8 at 2021-06-16 17:51:26.919860
cam 192900073 captured image of format BayerRG8 at 2021-06-16 17:51:26.965369
cam 101936 captured image of format Mono8 at 2021-06-16 17:51:27.310368
cam 101934 captured image of format Mono8 at 2021-06-16 17:51:27.325369
cam 101935 captured image of format Mono8 at 2021-06-16 17:51:27.355368
cam 101933 captured image of format Mono8 at 2021-06-16 17:51:27.360367
cam 192900073 captured image of format BayerRG8 at 2021-06-16 17:51:27.361368
cam 101936 captured image of format Mono8 at 2021-06-16 17:51:27.700897
cam 101934 captured image of format Mono8 at 2021-06-16 17:51:27.711897
cam 101935 captured image of format Mono8 at 2021-06-16 17:51:27.760904
cam 101933 captured image of format Mono8 at 2021-06

In [20]:
imager._stop_acquisitions()

In [8]:
imager.clear_all()


In [22]:
imager.cams[-1].remote_device.node_map.ExposureAuto.value = 'Off'

'Off'

In [10]:
import numpy as np
np.array(raw_images[0]).shape

(5, 2048, 2448)

In [11]:
num_frames =10

arr = np.empty((num_frames,5,2048,2448), dtype='uint8')
for frame_num in range(num_frames):
    arr[frame_num] = np.array(raw_images[frame_num])

In [25]:
import pickle
pickle.dump(raw_images, open("dump3.pkl", 'wb'))

In [14]:
np.save('arr', arr)